# Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import json
from itertools import islice # for slicing and dicing JSON records
import os

In [ ]:
def get_data(json_filename = 'arxiv-metadata-oai-snapshot.json', data_root = '/content/drive/MyDrive/카이스트/23봄/CS372/project'):
  with open(data_root + '/' + 'dataset' + '/' + json_filename, "rb") as f:
    for line in f:
      yield line

data_gen = get_data()

In [ ]:
def get_records(data_gen, chunksize=500):
    return [json.loads(record) for record in islice(data_gen, chunksize)]

records_per_chunk = 250000
data_records = get_records(data_gen, records_per_chunk)

In [ ]:
def split_records(data_records, num_profiles=100, random_state=42):
    np.random.seed(random_state)
    np.random.shuffle(data_records)
    train_records, test_records = data_records[:num_profiles], data_records[num_profiles:]
    return train_records, test_records

# Splitting the fetched records into train and test records
train_records, test_records = split_records(data_records, num_profiles=500)

In [ ]:
# Utility method to generate dataframe from list of dictionaries
def get_dataframe(list_of_dicts, columns=None):
    data = pd.DataFrame(list_of_dicts)
    if columns:
        data.columns = columns
    return data

# Generating dataframes for train and test records
train_df = get_dataframe(train_records)
test_df = get_dataframe(test_records)

In [ ]:
# Utility method to filter out certain features which are of use
def filter_features(data, features):
    return data[features]

# Filtering the test dataframes for features we selected
features = ['title', 'categories', 'abstract', 'update_date']
train_df = filter_features(train_df, features)
test_df = filter_features(test_df, features)

In [ ]:
# define the corpus to pull from
train_corpus = train_df['abstract'].head(10000)
test_corpus = test_df['abstract'].head(10000)

In [ ]:
train_df.head()

,title,categories,abstract,update_date
0,Probing FSR star cluster candidates in bulge/d...,astro-ph,We analyse 20 star cluster candidates projec...,2012-04-02
1,Further Development of the Tetron Model,hep-ph,After a prologue which clarifies some issues...,2014-11-18
2,Characterisation of a three-dimensional Browni...,physics.atom-ph,We present here a detailed study of the beha...,2007-09-18
3,Nonmagnetic Impurity Resonances as a Signature...,cond-mat.supr-con,The low energy band structure of the FeAs ba...,2013-05-29
4,Analysis of optical properties of strained sem...,cond-mat.mes-hall,Using multiband k*p theory we study the size...,2010-02-11


In [ ]:
test_df.head()

,title,categories,abstract,update_date
0,Strichartz Estimates for Water Waves,math.AP,In this paper we investigate the dispersive ...,2010-02-02
1,Analysis of Two Eclipsing Hot Subdwarf Binarie...,astro-ph.EP astro-ph.SR,"The formation of hot subdwarf stars (sdBs), ...",2015-03-18
2,A Bayesian reassessment of nearest-neighbour c...,stat.CO math.ST stat.TH,The k-nearest-neighbour procedure is a well-...,2008-02-12
3,"Clusters, Graphs, and Networks for Analysing I...",cs.AI cs.LG,"The proposal is to use clusters, graphs and ...",2007-07-11
4,"On the Capacity of the Heat Channel, Waterfill...",cs.IT math.IT,The heat channel is defined by a linear time...,2014-01-28


In [ ]:
# train_df.to_csv('/content/drive/MyDrive/cs372/project/dataset/train_df.csv', index = False)
# test_df.to_csv('/content/drive/MyDrive/cs372/project/dataset/test_df.csv', index = False)

## Removing unnecessary words

In [ ]:
import nltk
nltk.download('book')
from nltk.book import *
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to /root/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to /root/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package dependency_treebank is already up-to-date!
[nltk_data]    | Downloadi

In [ ]:
def tokenize_POS(paragraph):
  words = word_tokenize(paragraph)
  tagged_words = nltk.pos_tag(words)

  # Remove not important types of words
  excluded_tags = ['CC', 'DT', 'IN', 'TO', 'PRP', 'PRP$', 'MD', 'WP', 'WP$', 'WRB']
  filtered_words = [word for word, pos in tagged_words if pos not in excluded_tags]

  return ' '.join(filtered_words)

In [ ]:
# define the corpus to pull from
train_corpus = train_df['abstract'].head(10000)
test_corpus = test_df['abstract'].head(10000)

In [ ]:
train_corpus[0]

'  We analyse 20 star cluster candidates projected mostly in the bulge direction\n($|\\ell|<60^\\circ$). The sample contains all candidates in that sector\nclassified by \\citet{FSRcat} with quality flags denoting high probability of\nbeing star clusters. Bulge contamination in the colour-magnitude diagrams\n(CMDs) is in general important, while at lower Galactic latitudes disk stars\ncontribute as well. Properties of the candidates are investigated with 2MASS\nCMDs and stellar radial density profiles (RDPs) built with field star\ndecontaminated photometry. To uncover the nature of the structures we\ndecontaminate the CMDs from field stars using tools that we previously\ndeveloped to deal with objects in dense fields. We confirm in all cases\nexcesses in the RDPs with respect to the background level, as expected from the\nmethod the candidates were originally selected. CMDs and RDPs taken together\nrevealed 6 open clusters, 5 uncertain cases that require deeper observations,\nwhile 9 o

In [ ]:
train_corpus = pd.Series([tokenize_POS(abstract) for abstract in train_corpus])

In [ ]:
train_corpus[0]

'analyse 20 star cluster candidates projected mostly bulge direction ( $ |\\ell| < 60^\\circ $ ) . sample contains candidates sector classified \\citet { FSRcat } quality flags denoting high probability being star clusters . Bulge contamination colour-magnitude diagrams ( CMDs ) is general important , lower Galactic latitudes disk stars contribute as well . Properties candidates are investigated 2MASS CMDs stellar radial density profiles ( RDPs ) built field star decontaminated photometry . uncover nature structures decontaminate CMDs field stars using tools previously developed deal objects dense fields . confirm cases excesses RDPs respect background level , expected method candidates were originally selected . CMDs RDPs taken together revealed 6 open clusters , 5 uncertain cases that require deeper observations , 9 objects are possibly field density fluctuations .'

In [ ]:
train_df_doc = train_df

train_df_doc['abstract'] = train_df_doc['abstract'].apply(tokenize_POS)

# TF-IDF

In [ ]:
import time # for getting the runtime of cells
from sklearn.feature_extraction.text import TfidfVectorizer # for building word representations
from sklearn.metrics.pairwise import cosine_similarity # for getting similarity metrics
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def get_recommendations_TFIDF(abstract):
  abstract = tokenize_POS(abstract)
  corpus = pd.concat([train_corpus, pd.Series(abstract)], ignore_index=True)

  # Initialize an instance of tf-idf Vectorizer
  tfidf_vectorizer = TfidfVectorizer()
  # Generate the tf-idf vectors for the corpus
  tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

  # compute and print the cosine similarity matrix
  cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

  # Get the pairwise similarity scores
  sim_scores = list(enumerate(cosine_sim[-1]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # Check if the first result is the input abstract
  if corpus[int(sim_scores[0][0])].split() == abstract.split() and corpus[int(sim_scores[1][0])].split() == abstract.split():
    print(corpus[int(sim_scores[0][0])].split() == abstract.split())
    print(corpus[int(sim_scores[1][0])].split() == abstract.split())
    paper_indices = int(sim_scores[2][0])
    similarity = "{:.2f}%".format(sim_scores[2][1] * 100)  # Format similarity as a string with two decimal places and a percentage sign
  elif sim_scores[0][0] == 500:
    paper_indices = int(sim_scores[1][0])
    similarity = "{:.2f}%".format(sim_scores[1][1] * 100)  # Format similarity as a string with two decimal places and a percentage sign
  else:
    paper_indices = int(sim_scores[0][0])
    similarity = "{:.2f}%".format(sim_scores[0][1] * 100)  # Format similarity as a string with two decimal places and a percentage sign

  title = train_df['title'].iloc[paper_indices]
  categories = train_df['categories'].iloc[paper_indices]
  abstract = train_df['abstract'].iloc[paper_indices]
  return title, categories, abstract, similarity

In [ ]:
get_recommendations_TFIDF('''
Following line research have developed several years , argue best strategy understanding quantum gravity is build picture physical world notion time plays role . summarize here point view , explaining think fundamental description nature `` forget time '' , be done classical quantum theory . idea is develop formalism that treats dependent independent variables same footing . short , propose interpret mechanics theory relations variables , rather theory evolution variables time .
''')

('"Forget time"',
 'gr-qc',
 "Following line research have developed several years , argue best strategy understanding quantum gravity is build picture physical world notion time plays role . summarize here point view , explaining think fundamental description nature `` forget time '' , be done classical quantum theory . idea is develop formalism that treats dependent independent variables same footing . short , propose interpret mechanics theory relations variables , rather theory evolution variables time .",
 '100.00%')

# Doc2Vec

In [ ]:
import time
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

def train_doc2vec_model(corpus, vector_size=100, window=5, min_count=1, epochs=100):
    # create tagged document object
    tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate(corpus)] # words=word_tokenize(doc.lower())

    # initialize doc2vec model
    model = Doc2Vec(vector_size=vector_size, window=window, min_count=min_count, epochs=epochs)

    # build vocabulary
    model.build_vocab(tagged_data)

    # train doc2vec model
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

doc2vec_model = train_doc2vec_model(train_corpus)

In [ ]:
def get_recommendations_Doc2Vec(abstract):
    # remove unnecessary words
    abstract = tokenize_POS(abstract)
    # train the model
    model = doc2vec_model

    # infer the vector for the given abstract
    abstract_vector = model.infer_vector(abstract.split())

    # get the most similar abstract
    most_similar = model.dv.most_similar([abstract_vector], topn=2)

    # Check if the first result is the input abstract
    if train_df_doc['abstract'].iloc[int(most_similar[0][0])].split() == abstract.split():
        # print('True')
        paper_indices = int(most_similar[1][0])
        similarity = "{:.2f}%".format(most_similar[1][1] * 100)  # Format similarity as a string with two decimal places and a percentage sign
    else:
        # print('False')
        paper_indices = int(most_similar[0][0])
        similarity = "{:.2f}%".format(most_similar[0][1] * 100)  # Format similarity as a string with two decimal places and a percentage sign

    # Retrieve the details of the most similar abstract
    title = train_df['title'].iloc[paper_indices]
    categories = train_df['categories'].iloc[paper_indices]
    abstract = train_df['abstract'].iloc[paper_indices]
    return title, categories, abstract, similarity

# Deploy

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gradio as gr

def greet(paper):
    title_tfidf, categories_tfidf, abstract_tfidf, similarity_tfidf = get_recommendations_TFIDF(paper)
    title_doc2vec, categories_doc2vec, abstract_doc2vec, similarity_doc2vec = get_recommendations_Doc2Vec(paper)
    return title_tfidf, categories_tfidf, abstract_tfidf, title_doc2vec, categories_doc2vec, abstract_doc2vec

title = '''SimSearch:\n
A Similarity Search Tool for Research Paper Abstracts using NLP'''

demo = gr.Interface(
    title = title,
    fn = greet,
    inputs=gr.Textbox(placeholder='Abstract Here'),
    outputs=[gr.outputs.Textbox(label='''TFIDF Title'''),
             gr.outputs.Textbox(label='TFIDF Categories'),
             gr.outputs.Textbox(label='TFIDF Abstract'),
            #  gr.outputs.Textbox(label='Similarity')
             gr.outputs.Textbox(label='''Doc2Vec Title'''),
             gr.outputs.Textbox(label='Doc2Vec Categories'),
             gr.outputs.Textbox(label='Doc2Vec Abstract'),
             ],
    examples = [
        # ['''In this paper we consider permutations of sequences of partitions, obtaining\na result which parallels von Neumann's theorem on permutations of dense\nsequences and uniformly distributed sequences of points.\n'''],
                ['''A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to-leading logarithmic accuracy. The region of phase space is specified in which the calculation is most reliable. Good agreement is demonstrated with data from the Fermilab Tevatron, and predictions are made for more detailed tests with CDF and DO data. Predictions are shown for distributions of diphoton pairs produced at the energy of the Large Hadron Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs boson are contrasted with those produced from QCD processes at the LHC, showing that enhanced sensitivity to the signal can be obtained with judicious selection of events.'''],
                ['''Most physical experiments are usually described repeated measurements random variables . experimental data registered on-line computers form time series outcomes . frequencies different outcomes are compared probabilities provided algorithms quantum theory ( QT ) . spite statistical predictions QT claim was made theory provided most complete description data underlying physical phenomena . claim be easily rejected fine structures , averaged out standard statistical descriptive analysis , were found time series experimental data . search structures one has use more subtle statistical tools which were developed study time series produced various stochastic processes . talk review tools . example show standard descriptive statistical analysis data is unable reveal fine structure simulated sample AR ( 2 ) stochastic process . emphasize once again violation Bell inequalities gives information completeness non locality QT . appropriate way test completeness quantum theory is search fine structures time series experimental data means purity tests studying autocorrelation partial autocorrelation functions .''']]
    )

demo.launch(share=True, debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ebee077e3938e48120.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ebee077e3938e48120.gradio.live
